In [40]:
from faker import Faker
import json
import sqlite3
import random
import secrets
from datetime import date

fake = Faker(['en_US'])

In [41]:
# generate fake products name
def generate_fake_products_name():
    return fake.bs().title()

# generate fake products quantity
def generate_fake_products_quantity():
    return random.randint(1, 250)

# generate products sku units ratio
def generate_fake_products_sku_units_ratio():
    return random.randint(1, 24)

# generate fake description
def generate_fake_description():
    while True:
        description = fake.text()
        if description.count('.') > 1:
            return description.title()


# generate fake brand
def generate_fake_brand():
    return fake.company()

# generate fake weight
def generate_fake_weight():
    return round(random.uniform(0.1, 15.0), 2)

# generate fake dimensions
def generate_fake_dimensions():
    return f"{random.randint(1, 100)}x{random.randint(1, 100)}x{random.randint(1, 100)}"

# generate fake date added
def generate_fake_date_added():
    return fake.date()

# generate fake image URL
def generate_fake_image_url():
    width = random.randint(100, 1000)
    height = random.randint(100, 1000)
    return f"https://picsum.photos/{width}/{height}"

# crypto id generator
def generate_crypto_id():
    return secrets.token_hex(36)


In [42]:
# fake b2b price
def generate_fake_b2b_price():
    return round(random.uniform(1, 1000.0), 2)

# fake b2c price
def generate_fake_b2c_price():
    return round(random.uniform(1, 1000.0), 2)

# fake start date
def generate_fake_start_date():
    return fake.date_this_year().strftime("%Y-%m-%d")

# fake end date
def generate_fake_end_date():
    return fake.date_between(date(2025, 1, 1), date(2025, 12, 31)).strftime("%Y-%m-%d")

def fake_pricings(id, product_name, sku_ratio):
    fake_pricings = {
        "id": secrets.token_hex(36),
        "product_id": id,
        "product_name": product_name,
        "sku_units_ratio": sku_ratio,
        "b2b_price": generate_fake_b2b_price(),
        "b2c_price": generate_fake_b2c_price(),
        "start_date": generate_fake_start_date(),
        "end_date": generate_fake_end_date()
    }
    
    return fake_pricings
    
print(generate_fake_end_date())

2025-08-09


In [43]:
def fake_products():
    crypto_id = generate_crypto_id()
    product_name = generate_fake_products_name()
    sku_ratio = generate_fake_products_sku_units_ratio()
    
    fake_products = {
        "id": crypto_id,
        "name": product_name,
        "quantity": generate_fake_products_quantity(),
        "sku_units_ratio": sku_ratio,
        "description": generate_fake_description(),
        "brand": generate_fake_brand(),
        "weight": generate_fake_weight(),
        "dimensions": generate_fake_dimensions(),
        "date_added": generate_fake_date_added(),
        "image_url": generate_fake_image_url()
    }
    
    pricings = fake_pricings(crypto_id, product_name, sku_ratio)
    
    return fake_products, pricings

def multiple_fake_products_pricings(n):
    fake_products_data = []
    fake_pricings_data = []

    for _ in range(n):
        product, pricing = fake_products()
        fake_products_data.append(product)
        fake_pricings_data.append(pricing)
                                  
    return json.dumps(fake_products_data, indent=4), json.dumps(fake_pricings_data, indent=4)

# test
x, y = multiple_fake_products_pricings(1)
print(x)
print(y)

[
    {
        "id": "23148832b0a0ad904a1d4373f7097146d1e368f10f8d443f0f4c61c035848731898d8b89",
        "name": "Synthesize Rich Experiences",
        "quantity": 94,
        "sku_units_ratio": 19,
        "description": "Them Their Order Office Mrs Far Community. Model Deal Spring Share Product Father.",
        "brand": "Allen-Wilson",
        "weight": 11.75,
        "dimensions": "50x11x61",
        "date_added": "2003-12-15",
        "image_url": "https://picsum.photos/472/902"
    }
]
[
    {
        "id": "6371b7f3027df413fca45cf1a36dd69aeaaa222f1fb08c3e854a0ca3281509eb1b0a254e",
        "product_id": "23148832b0a0ad904a1d4373f7097146d1e368f10f8d443f0f4c61c035848731898d8b89",
        "product_name": "Synthesize Rich Experiences",
        "sku_units_ratio": 19,
        "b2b_price": 858.25,
        "b2c_price": 123.14,
        "start_date": "2024-05-07",
        "end_date": "2025-12-17"
    }
]


In [44]:
# insert fake products data into database
conn = sqlite3.connect('database/database.db')
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS products (
        id TEXT PRIMARY KEY NOT NULL,
        name TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        sku_units_ratio INTEGER NOT NULL,
        description TEXT NOT NULL,
        brand TEXT NOT NULL,
        weight REAL NOT NULL,
        dimensions TEXT NOT NULL,
        date_added DATE NOT NULL,
        image_url TEXT NOT NULL
    )
    '''
)

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS pricings (
        id TEXT PRIMARY KEY NOT NULL,
        product_id TEXT NOT NULL,
        product_name TEXT NOT NULL,
        sku_units_ratio INTEGER NOT NULL,
        b2b_price REAL NOT NULL,
        b2c_price REAL NOT NULL,
        start_date DATE NOT NULL,
        end_date DATE NOT NULL
    )
    '''
)

product_json_data, pricings_json_data = multiple_fake_products_pricings(250)

# save fake products data into json file with 4 spaces indentation
with open('json_data/products.json', 'w') as file:
    file.write(product_json_data)

# save fake pricings data into json file with 4 spaces indentation
with open('json_data/pricings.json', 'w') as file:
    file.write(pricings_json_data)
    
    
for products in json.loads(product_json_data):
    c.execute(
        '''
        INSERT INTO products (id, name, quantity, sku_units_ratio, description, brand, weight, dimensions, date_added, image_url)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''',
        (
            products['id'],
            products['name'],
            products['quantity'],
            products['sku_units_ratio'],
            products['description'],
            products['brand'],
            products['weight'],
            products['dimensions'],
            products['date_added'],
            products['image_url']
        )
    )
    
for pricings in json.loads(pricings_json_data):
    c.execute(
        '''
        INSERT INTO pricings (id, product_id, product_name, sku_units_ratio, b2b_price, b2c_price, start_date, end_date)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''',
        (
            pricings['id'],
            pricings['product_id'],
            pricings['product_name'],
            pricings['sku_units_ratio'],
            pricings['b2b_price'],
            pricings['b2c_price'],
            pricings['start_date'],
            pricings['end_date']
        )
    )
    
conn.commit()
conn.close() # must close the connection after commiting the changes

In [45]:
# delete the products table
# establish connection
#conn = sqlite3.connect('database/database.db')
#c = conn.cursor()
#c.execute('DROP TABLE products')
#c.execute('DROP TABLE pricings')
#conn.commit()